In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup 

In [2]:
UA = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.3'

In [3]:
headers = { 'User-Agent': UA }

In [4]:
res = requests.get('https://zozo.jp/_help/', headers=headers)
res.encoding = res.apparent_encoding
soup = BeautifulSoup(res.text, 'html.parser')

In [5]:
page_ids = []
for a_tag in soup.select('.accordion ul li a'):
    page_id = a_tag.attrs.get('href').split('=')[-1]
    page_ids += [page_id]

In [6]:
len(page_ids)

152

In [7]:
def get_page_content(page_id):
    res = requests.get('https://helpfeel.com/zozo-help/null-{}'.format(page_id), headers=headers)
    res.encoding = res.apparent_encoding
    soup = BeautifulSoup(res.text, 'html.parser')
    return soup.select_one('.page-content').text.strip()

In [8]:
results = []
for page_id in page_ids:
    url = 'https://zozo.jp/_help/default.html?id={}'.format(page_id)
    content = get_page_content(page_id)

    results += [{
        'url': url,
        'content': content,
    }]

In [10]:
df = pd.DataFrame(results)
df.head(5)

,url,content
0,https://zozo.jp/_help/default.html?id=5ece39d3...,商品のサイズ商品のサイズは、商品ページ下部にある「サイズ」の項目をご覧ください。当サイトでは...
1,https://zozo.jp/_help/default.html?id=5ece39d3...,商品のカラー・デザイン商品の画像は、できる限り実際の商品に近づけた状態で掲載をしております。...
2,https://zozo.jp/_help/default.html?id=5ece47a3...,商品の販売価格サイトの表示価格は、販売ショップがZOZOTOWNで販売するために設定した価格...
3,https://zozo.jp/_help/default.html?id=5ece39d4...,ZOZOUSEDの取扱い商品ZOZOUSEDでは、買取サービスや買い替え割・いつでも買い替え...
4,https://zozo.jp/_help/default.html?id=5ece39d4...,ZOZOUSED商品の掲載画像撮影時の照明やカメラの角度、お客様の端末設定により、実際の色味...


In [11]:
df.to_csv('./data/raw_content.csv', index=False)